<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/305_HITL_QueueSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

@app.get("/api/v1/pending-reviews")
async def get_pending_reviews_endpoint(
    human_role: Optional[str] = None,
    limit: Optional[int] = None
):
    """
    Get pending reviews from the queue.

    Query parameters:
    - human_role: Filter by human role (optional)
    - limit: Maximum number of reviews to return (optional)
    """
    try:
        reviews = get_pending_reviews(human_role=human_role, limit=limit)
        return {
            "pending_reviews": reviews,
            "count": len(reviews)
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")


@app.get("/api/v1/pending-reviews/{task_id}")
async def get_pending_review_endpoint(task_id: str):
    """
    Get a specific pending review by task_id.
    """
    try:
        review = get_pending_review(task_id)
        if not review:
            raise HTTPException(status_code=404, detail=f"Pending review for task {task_id} not found")
        return review
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")


class HumanReviewRequest(BaseModel):
    """Human review request model"""
    human_role: str
    human_decision: str = Field(..., description="'approve', 'reject', 'override', or 'modify'")
    human_feedback: Optional[str] = None
    confidence_assessment: Optional[str] = Field(None, description="'high', 'medium', or 'low'")


@app.post("/api/v1/pending-reviews/{task_id}/review")
async def submit_review_endpoint(task_id: str, review: HumanReviewRequest):
    """
    Submit a human review for a pending task.
    """
    try:
        # Get pending review
        pending = get_pending_review(task_id)
        if not pending:
            raise HTTPException(status_code=404, detail=f"Pending review for task {task_id} not found")

        # Create human review dict
        human_review_dict = {
            "review_id": f"review_{task_id}",
            "task_id": task_id,
            "human_role": review.human_role,
            "human_decision": review.human_decision,
            "human_feedback": review.human_feedback or "",
            "confidence_assessment": review.confidence_assessment or "medium",
            "timestamp": datetime.now().isoformat()
        }

        # Map human decision to final decision
        decision_map = {
            "approve": "approved",
            "reject": "rejected",
            "override": "override_approved",
            "modify": "modified_and_approved"
        }

        final_decision_dict = {
            "task_id": task_id,
            "final_decision": decision_map.get(review.human_decision, "approved"),
            "decision_source": "human",
            "confidence_score": pending.get("confidence_score", 0.0),
            "risk_level": pending.get("risk_level", "low"),
            "human_involved": True,
            "latency_seconds": 0.0
        }

        # Update review status
        updated = update_review_status(
            task_id=task_id,
            human_review=human_review_dict,
            final_decision=final_decision_dict
        )

        return {
            "task_id": task_id,
            "status": "reviewed",
            "human_review": human_review_dict,
            "final_decision": final_decision_dict,
            "updated_entry": updated
        }

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")


@app.get("/api/v1/queue/stats")
async def get_queue_stats_endpoint():
    """
    Get queue statistics.
    """
    try:
        stats = get_queue_stats()
        return stats
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")




## 🧠 Big Picture: What This Section Is Doing

This code turns your **queue** from an internal data structure into a **human-facing workflow**.

Before this:

* the agent knew a human was needed
* tasks were queued internally

After this:

* humans can **see** what’s waiting
* humans can **act** on tasks
* the system can **accept human decisions live**

👉 This is what makes HITL *operational*, not theoretical.

---

## 🧩 Mental Model: This Is the “Human UI Backend”

Think of these endpoints as the **backend for a human review dashboard**.

They answer questions like:

* “What do I need to review?”
* “What’s the status of this task?”
* “Let me submit my decision”
* “How overloaded are we?”

You could plug these directly into:

* a web UI
* an internal tool
* a Slack bot
* a workflow system

---

## 🔍 What Each Endpoint Means (Conceptually)

### 1️⃣ `GET /pending-reviews`

**“Show me my inbox.”**

* Lists tasks waiting for humans
* Can filter by role
* Can limit results

This is how humans *pull work*.

---

### 2️⃣ `GET /pending-reviews/{task_id}`

**“Show me this specific case.”**

* Lets a human inspect details
* Used when clicking into a task

This supports careful review, not blind approval.

---

### 3️⃣ `POST /pending-reviews/{task_id}/review`

**“Here is my decision.”**

This is the most important endpoint.

It:

* records human judgment
* maps it to a final decision
* updates the queue
* preserves audit context

Conceptually:

> **This is where human authority is exercised.**

The agent *never* argues back.

---

### 4️⃣ `GET /queue/stats`

**“How busy are we?”**

This gives:

* backlog size
* role-based load
* operational health signals

This is for:

* managers
* ops teams
* system monitoring

---

## 🧠 Why This Is Not “Extra Boilerplate”

This part is not about APIs anymore — it’s about **workflow realism**.

Without these endpoints:

* HITL looks good on paper
* humans have no way to interact
* the system can’t scale

With them:

* humans and AI share responsibility
* work flows across time
* accountability is explicit

---

## 🎯 The Key Insight (Most Important)

> **Human-in-the-Loop is not a decision problem.
> It’s a coordination problem.**

These endpoints solve coordination:

* who sees what
* when
* how decisions enter the system

That’s why this section matters.



# Test queue system for HITL Orchestrator

In [ ]:
"""Test queue system for HITL Orchestrator"""

from agents.hitl_orchestrator.utilities.queue import (
    add_to_queue,
    get_pending_reviews,
    get_pending_review,
    update_review_status,
    get_review_history,
    get_queue_stats,
    clear_queue
)
from agents.hitl_orchestrator.api import process_task_request
from datetime import datetime


def test_add_to_queue():
    """Test adding tasks to queue"""
    clear_queue()  # Start fresh

    task_data = {
        "task": {
            "task_id": "queue_test_001",
            "task_type": "document_classification",
            "task_description": "Test task",
            "domain": "customer_support",
            "risk_level": "medium"
        },
        "agent_output": {
            "predicted_label": "Test Label",
            "explanation": "Test explanation"
        },
        "confidence_score": 0.65,
        "risk_level": "medium"
    }

    routing_decision = {
        "task_id": "queue_test_001",
        "routing_decision": "human_review",
        "assigned_human_role": "domain_reviewer",
        "rule_applied": "rule_003"
    }

    entry = add_to_queue("queue_test_001", task_data, routing_decision)

    assert entry["task_id"] == "queue_test_001"
    assert entry["status"] == "pending"
    assert entry["assigned_human_role"] == "domain_reviewer"
    assert "queued_at" in entry

    print("\n✅ Add to queue test passed!")


def test_get_pending_reviews():
    """Test retrieving pending reviews"""
    clear_queue()

    # Add multiple tasks
    for i in range(3):
        task_data = {
            "task": {"task_id": f"queue_test_{i:03d}", "risk_level": "medium"},
            "agent_output": {},
            "confidence_score": 0.65,
            "risk_level": "medium"
        }
        routing_decision = {
            "task_id": f"queue_test_{i:03d}",
            "routing_decision": "human_review",
            "assigned_human_role": "domain_reviewer" if i < 2 else "senior_manager"
        }
        add_to_queue(f"queue_test_{i:03d}", task_data, routing_decision)

    # Get all pending
    all_pending = get_pending_reviews()
    assert len(all_pending) == 3

    # Get by role
    domain_reviewer_pending = get_pending_reviews(human_role="domain_reviewer")
    assert len(domain_reviewer_pending) == 2

    # Get with limit
    limited = get_pending_reviews(limit=2)
    assert len(limited) == 2

    print("\n✅ Get pending reviews test passed!")
    print(f"   Total pending: {len(all_pending)}")
    print(f"   Domain reviewer pending: {len(domain_reviewer_pending)}")


def test_update_review_status():
    """Test updating review status"""
    clear_queue()

    # Add task to queue
    task_data = {
        "task": {"task_id": "queue_test_004", "risk_level": "medium"},
        "agent_output": {},
        "confidence_score": 0.65,
        "risk_level": "medium"
    }
    routing_decision = {
        "task_id": "queue_test_004",
        "routing_decision": "human_review",
        "assigned_human_role": "domain_reviewer"
    }
    add_to_queue("queue_test_004", task_data, routing_decision)

    # Submit review
    human_review = {
        "review_id": "review_004",
        "task_id": "queue_test_004",
        "human_role": "domain_reviewer",
        "human_decision": "approve",
        "human_feedback": "Looks good",
        "confidence_assessment": "high",
        "timestamp": datetime.now().isoformat()
    }

    final_decision = {
        "task_id": "queue_test_004",
        "final_decision": "approved",
        "decision_source": "human",
        "confidence_score": 0.65,
        "risk_level": "medium",
        "human_involved": True,
        "latency_seconds": 0.0
    }

    updated = update_review_status("queue_test_004", human_review, final_decision)

    assert updated["status"] == "reviewed"
    assert updated["human_review"]["human_decision"] == "approve"
    assert "reviewed_at" in updated

    # Check it's no longer in pending
    pending = get_pending_review("queue_test_004")
    assert pending is None  # Should be None since status is "reviewed"

    # Check it's in history
    history = get_review_history("queue_test_004")
    assert len(history) == 1
    assert history[0]["status"] == "reviewed"

    print("\n✅ Update review status test passed!")
    print(f"   Status: {updated['status']}")
    print(f"   Final Decision: {updated['final_decision']['final_decision']}")


def test_queue_stats():
    """Test queue statistics"""
    clear_queue()

    # Add some tasks
    for i in range(5):
        task_data = {
            "task": {"task_id": f"stats_test_{i:03d}", "risk_level": "medium"},
            "agent_output": {},
            "confidence_score": 0.65,
            "risk_level": "medium"
        }
        routing_decision = {
            "task_id": f"stats_test_{i:03d}",
            "routing_decision": "human_review",
            "assigned_human_role": "domain_reviewer" if i < 3 else "senior_manager"
        }
        add_to_queue(f"stats_test_{i:03d}", task_data, routing_decision)

    # Review one
    human_review = {
        "review_id": "review_stats_001",
        "task_id": "stats_test_001",
        "human_role": "domain_reviewer",
        "human_decision": "approve",
        "timestamp": datetime.now().isoformat()
    }
    final_decision = {
        "task_id": "stats_test_001",
        "final_decision": "approved",
        "decision_source": "human",
        "confidence_score": 0.65,
        "risk_level": "medium",
        "human_involved": True,
        "latency_seconds": 0.0
    }
    update_review_status("stats_test_001", human_review, final_decision)

    # Get stats
    stats = get_queue_stats()

    assert stats["total_pending"] == 4  # 5 - 1 reviewed
    assert stats["total_reviewed"] == 1
    assert stats["pending_by_role"]["domain_reviewer"] == 2  # 3 - 1 reviewed
    assert stats["pending_by_role"]["senior_manager"] == 2

    print("\n✅ Queue stats test passed!")
    print(f"   Total Pending: {stats['total_pending']}")
    print(f"   Total Reviewed: {stats['total_reviewed']}")
    print(f"   Pending by Role: {stats['pending_by_role']}")


def test_integration_with_api():
    """Test queue integration with process_task_request"""
    clear_queue()

    # Process a task that needs review
    task = {
        "task_id": "api_queue_test_001",
        "task_type": "document_classification",
        "task_description": "Test task",
        "domain": "customer_support",
        "risk_level": "medium",
        "timestamp": datetime.now().isoformat()
    }

    agent_output = {
        "predicted_label": "Test Label",
        "explanation": "Test explanation"
    }

    result = process_task_request(task, agent_output, confidence_score=0.65)

    # Should be pending review
    assert result["status"] == "pending_review"

    # Should be in queue
    pending = get_pending_review("api_queue_test_001")
    assert pending is not None
    assert pending["status"] == "pending"

    print("\n✅ API integration test passed!")
    print(f"   Task ID: {result['task_id']}")
    print(f"   Status: {result['status']}")
    print(f"   In Queue: {pending is not None}")


if __name__ == "__main__":
    print("Testing HITL Orchestrator Queue System...")
    print("=" * 60)

    test_add_to_queue()
    test_get_pending_reviews()
    test_update_review_status()
    test_queue_stats()
    test_integration_with_api()

    print("\n" + "=" * 60)
    print("✅ All queue system tests passed!")



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_004_HITL_ColabOrchestrator % python test_hitl_queue.py
Testing HITL Orchestrator Queue System...
============================================================

✅ Add to queue test passed!

✅ Get pending reviews test passed!
   Total pending: 3
   Domain reviewer pending: 2

✅ Update review status test passed!
   Status: reviewed
   Final Decision: approved

✅ Queue stats test passed!
   Total Pending: 4
   Total Reviewed: 1
   Pending by Role: {'domain_reviewer': 2, 'senior_manager': 2}

✅ API integration test passed!
   Task ID: api_queue_test_001
   Status: pending_review
   In Queue: True

============================================================


All queue tests passed. The queue system is working.

## What we've accomplished

### Completed enhancements

1. Real-time processing
   - Process tasks without file loading
   - Fast, direct function calls

2. HTTP API
   - RESTful endpoints
   - Production-ready API server
   - Auto-generated documentation

3. Queue system
   - Automatic queueing of tasks needing review
   - Role-based filtering
   - Review submission
   - Queue statistics

### Current capabilities

- Process tasks in real time
- Route based on confidence and risk
- Queue tasks for human review
- Track pending and reviewed tasks
- Submit reviews via API
- Monitor queue health

## Next enhancement options

1. Dynamic policy learning — learn from human feedback to improve routing
2. Confidence score calibration — improve accuracy per domain
3. Webhook notifications — notify reviewers when tasks need review
4. Database backend — replace in-memory queue with persistent storage
